#   뉴스 감성 분석

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

import re
import json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")


`(3) 뉴스 데이터 로드`

In [ ]:
# pickle 파일에서 데이터 로드
import pickle

with open("processed_news_articles.pkl", "rb") as f:
    loaded_docs = pickle.load(f)
    print(f"로드된 문서 수: {len(loaded_docs)}")

In [ ]:
loaded_docs[0].metadata

---

## **LLM을 이용한 뉴스 감성 분석**

- 특정 주제에 대한 언론 보도 경향 파악
- 시간에 따른 뉴스 감성 변화 트래킹
- 기업 또는 산업 관련 뉴스의 긍정/부정 비율 분석
- 사회적 이슈에 대한 미디어 반응 연구

---

### 1. **감성 분석(Sentiment Analysis)**

- **감성 분석**은 텍스트에 표현된 감정과 의견을 파악하는 기술임
- 감성 분석은 텍스트를 **긍정, 부정, 중립** 등으로 분류함
- 텍스트의 **맥락과 뉘앙스**를 고려한 감성 평가가 가능함

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict, Any, Literal
from pydantic import BaseModel, Field

# 감성 분석 결과를 위한 Pydantic 모델
class SentimentAnalysis(BaseModel):
    """뉴스 기사의 감성 분석 결과"""
    sentiment: Literal["긍정적", "부정적", "중립적"] = Field(description="감성 분석 결과 (긍정적, 부정적, 중립적)")
    score: float = Field(description="감성 점수 (0~1, 1에 가까울수록 긍정적)")
    key_phrases: List[str] = Field(description="기사에서 감성을 나타내는 주요 구문")
    explanation: str = Field(description="감성 분석 결과에 대한 설명")

# LLM 초기화
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)
structured_llm = llm.with_structured_output(SentimentAnalysis)

# 감성 분석 프롬프트 템플릿
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 뉴스 기사의 감성을 분석하는 전문가입니다. 
    주어진 뉴스 기사를 분석하여 긍정적, 부정적, 중립적 감성을 파악하세요.
    감성 점수는 0(매우 부정적)에서 1(매우 긍정적) 사이의 값으로 제공하세요.
    기사에서 감성을 나타내는 주요 구문을 추출하고 분석 결과를 설명하세요."""),
    ("human", "{news_article}")
])

# 감성 분석 체인 생성
sentiment_analysis_chain = sentiment_prompt | structured_llm

# 예시 뉴스 기사
news_article = """
한국 IT 기업, 글로벌 시장 진출 성공적

서울 - 국내 주요 IT 기업들이 글로벌 시장에서 큰 성과를 거두고 있다. 
지난 분기 대비 매출이 15% 증가하였으며, 특히 북미와 유럽 시장에서의 성장이 두드러졌다. 
업계 관계자는 "혁신적인 기술력과 차별화된 서비스가 좋은 평가를 받고 있다"고 밝혔다.
그러나 일부 전문가들은 글로벌 경쟁이 심화되는 상황에서 지속적인 기술 투자가 필요하다고 지적했다.
"""

# 감성 분석 실행
result = sentiment_analysis_chain.invoke({"news_article": news_article})
print(f"감성: {result.sentiment}")
print(f"점수: {result.score}")
print(f"주요 구문: {result.key_phrases}")
print(f"설명: {result.explanation}")

---

### **[실습]**

- 프롬프트를 수정하여 감성 분석 체인을 정의합니다. 
- 뉴스 데이터를 사용하여 감성 분석 작업을 수행합니다. 

In [ ]:
# 여기에 코드를 작성하세요. 

---

### 2. **데이터 증강(Data Augmentation)**

   - 감성 분석 모델 학습용 데이터 확장
   - 다양한 관점의 기사 생성을 통한 균형 잡힌 분석
   - 특정 주제에 대한 다각적 분석 자료 확보

In [ ]:
# 데이터 증강을 위한 Pydantic 모델
class AugmentedNews(BaseModel):
    """증강된 뉴스 기사"""
    headline: str = Field(description="뉴스 기사의 제목")
    content: str = Field(description="증강된 뉴스 기사 내용")
    perspective: str = Field(description="증강된 기사의 관점 또는 초점")

# 데이터 증강 프롬프트 템플릿
augmentation_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 뉴스 기사 작성 전문가입니다. 
    주어진 원본 뉴스 기사를 바탕으로 다음 지시에 따라 새로운 버전의 기사를 작성하세요:
    
    1. 원본 기사의 사실과 정보를 유지하면서 표현 방식과 구조를 변경하세요.
    2. 제시된 관점이나 초점(예: 경제적 영향, 사회적 의미, 기술적 측면 등)에 맞게 내용을 재구성하세요.
    3. 기사의 전체적인 톤을 지정된 방향(예: 중립적, 분석적, 설명적)으로 조정하세요.
    4. 적절한 제목을 생성하세요.
    
    출력은 구조화된 형식으로 제공하되, 내용이 자연스럽고 실제 뉴스 기사처럼 읽히도록 하세요."""),
    ("human", """
    원본 기사: {original_news}
    
    증강 지시사항:
    - 관점/초점: {perspective}
    - 톤: {tone}
    """)
])

# 구조화된 출력을 위한 LLM 설정
augmentation_llm = llm.with_structured_output(AugmentedNews)

# 데이터 증강 체인 생성
data_augmentation_chain = augmentation_prompt | augmentation_llm

# 다양한 관점과 톤으로 뉴스 기사 증강
perspectives = ["경제적 영향", "사회적 의미", "기술적 측면", "국제 관계"]
tones = ["중립적", "분석적", "설명적"]

# 원본 뉴스 기사로 여러 버전 생성
augmented_news_list = []

for perspective in perspectives:
    for tone in tones:
        augmented = data_augmentation_chain.invoke({
            "original_news": news_article,
            "perspective": perspective,
            "tone": tone
        })
        augmented_news_list.append(augmented)
        print(f"\n=== {perspective} / {tone} 관점 증강 결과 ===")
        print(f"제목: {augmented.headline}")
        print(f"내용 일부: {augmented.content[:150]}...")

print(f"\n총 {len(augmented_news_list)}개의 증강된 뉴스 기사가 생성되었습니다.")

---

### **[실습]**

- 프롬프트를 수정하여 데이터 증강 체인을 정의합니다. 
- 뉴스 데이터를 사용하여 데이터 증강 작업을 수행합니다. 

In [ ]:
# 여기에 코드를 작성하세요. 